In [1]:
import torch 
import torchmetrics
import torch.nn as nn
from pytorch_lightning import LightningModule,Trainer
from torch.utils.data import TensorDataset,DataLoader

In [2]:
from EEG_inception_dataset import EEG_inception
from model import Conv1D_v2

### chrononet model

In [3]:
import torch
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, inplace):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=inplace, out_channels=16, kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv1d(in_channels=inplace, out_channels=16, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv1d(in_channels=inplace, out_channels=16, kernel_size=8, stride=2, padding=3)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x1 = self.relu(self.conv1(x))
        x2 = self.relu(self.conv2(x))
        x3 = self.relu(self.conv3(x))
        x = torch.cat([x1, x2, x3], dim=1)
        return x

class ChronoNet(nn.Module):
    def __init__(self, channel):
        super().__init__()
        # Use fewer blocks and channels
        self.block1 = Block(channel)
        self.block2 = Block(48)  # Adjust input to match output channels from Block
        
        # Use a single GRU layer to simplify the model
        self.gru = nn.GRU(input_size=48, hidden_size=32, batch_first=True)
        
        # Linear layer for output prediction
        self.fc = nn.Sequential(
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = x.permute(0, 2, 1)  # Adjust for GRU input
        
        # GRU forward pass
        gru_out, _ = self.gru(x)
        # Use the last output from GRU
        x = gru_out[:, -1, :]  
        
        # Fully connected layer
        x = self.fc(x)
        return x


In [4]:
input=torch.randn(3,8,1000)
input.shape
model=ChronoNet(8)
out=model(input)
out.shape
print(torch.sigmoid(out))


tensor([[0.4306],
        [0.4347],
        [0.4354]], grad_fn=<SigmoidBackward0>)


### instantiating the lighting module for the model 

In [5]:
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torchmetrics

class ChronoModel(LightningModule):
    def __init__(self, attribute):
        super(ChronoModel, self).__init__()
        self.attribute = attribute
        self.model = attribute["model"] # initialize the model
        self.lr = 1e-4
        self.bs = 64
        self.worker = 2
        self.acc = torchmetrics.Accuracy(task="binary")
     
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        dataset = self.attribute["train_dataset"]
        return DataLoader(dataset, batch_size=self.bs, num_workers=self.worker, shuffle=True)

    def val_dataloader(self):
        dataset = self.attribute["val_dataset"]
        return DataLoader(dataset, batch_size=self.bs, num_workers=self.worker, shuffle=False)

    def training_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        preds = (torch.sigmoid(out.flatten()) > 0.5).long()
        # print(preds.)
        acc = self.acc(preds, label.long().flatten())
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        preds = (torch.sigmoid(out.flatten()) > 0.5).long()
        acc = self.acc(preds, label.long().flatten())
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def on_train_epoch_end(self):
        # Retrieve the average training loss
        train_loss = self.trainer.callback_metrics.get("train_loss_epoch", None)
        if train_loss is not None:
            self.print(f"Epoch {self.current_epoch} - Training Loss: {train_loss:.4f}")

    def on_validation_epoch_end(self):
        # Retrieve the average validation loss
        val_loss = self.trainer.callback_metrics.get("val_loss", None)
        if val_loss is not None:
            self.print(f"Epoch {self.current_epoch} - Validation Loss: {val_loss:.4f}")


In [6]:
attributes = {1 : {"model" : ChronoNet(channel=8), "train_dataset" : EEG_inception(kind="train", normalize= 1, balancing="equal_samples"), "val_dataset" : EEG_inception(kind = "val", normalize= 1)}}

model=ChronoModel(attribute = attributes[1])

should be here 


100%|██████████| 4988/4988 [00:32<00:00, 153.97it/s]


(4988, 8, 750) shap[e]


100%|██████████| 7650/7650 [00:11<00:00, 661.10it/s] 


(4988, 8, 750) in here dataset
train main_job done 10790 4988 4988
should be here 


100%|██████████| 856/856 [00:07<00:00, 107.94it/s]


(856, 8, 750) shap[e]


100%|██████████| 7650/7650 [00:04<00:00, 1850.53it/s]


(856, 8, 750) in here dataset
val main_job done 1350 856 856


In [7]:
trainer=Trainer(max_epochs=150)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model)

You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | ChronoNet         | 21.1 K | train
1 | acc       | BinaryAccuracy    | 0      | train
2 | criterion | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------
21.1 K    Trainable params
0         Non-trainable params
21.1 K    Total params
0.084     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\zokov\.conda\envs\py3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Epoch 0 - Validation Loss: 0.6934


c:\Users\zokov\.conda\envs\py3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


...

Epoch 26 - Validation Loss: 0.7139
Epoch 26 - Training Loss: 0.6753


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 27 - Validation Loss: 0.7155
Epoch 27 - Training Loss: 0.6738


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 28 - Validation Loss: 0.7165
Epoch 28 - Training Loss: 0.6731


In [ ]:
print(trainer.callback_metrics)

{'train_loss': tensor(0.6919), 'train_loss_step': tensor(0.7212), 'train_acc': tensor(0.5230), 'train_acc_step': tensor(0.3750), 'val_loss': tensor(0.6909), 'val_acc': tensor(0.5275), 'train_loss_epoch': tensor(0.6919), 'train_acc_epoch': tensor(0.5230)}


In [ ]:
train_data = EEG_inception(kind="train", normalize = False)
dl = DataLoader(train_data, batch_size=64, num_workers=2, shuffle=True)

(1000, 8, 1000) in here dataset
train kind 1000 1000 1000
main_job done


In [ ]:
pred_label = []
true_label = []

device = torch.device("cpu")
for ii, (data, label) in enumerate(dl):
    input = data.float().to(device)
    label = label.to(device)

    pred = model(input).float()
    pred = torch.sigmoid(pred)
    pred = (pred >= 0.5).float().to(device).data

    pred = pred.view(-1)
    
    pred_label.append(pred)
    true_label.append(label)
    # print(pred, "val")
    # print(label)
pred_label = torch.cat(pred_label, 0)
true_label = torch.cat(true_label, 0)

val_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)

print(val_accuracy)

tensor(0.5210)


In [ ]:
num_ones = torch.sum(pred_label == 0).item()

In [ ]:
num_ones

795

In [ ]:
torch.sum(pred_label == 1).item()

2559

In [ ]:
########################################################################

In [ ]:
import torch
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, inplace):
        super().__init__()
        # Add batch normalization and proper initialization
        self.conv1 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=2, stride=2, padding=0)
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=8, stride=2, padding=3)
        self.bn3 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        # Initialize weights properly
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.kaiming_normal_(self.conv3.weight)
    
    def forward(self, x):
        x1 = self.dropout(self.relu(self.bn1(self.conv1(x))))
        x2 = self.dropout(self.relu(self.bn2(self.conv2(x))))
        x3 = self.dropout(self.relu(self.bn3(self.conv3(x))))
        x = torch.cat([x1, x2, x3], dim=1)
        return x

class ChronoNet(nn.Module):
    def __init__(self, channel):
        super().__init__()
        self.block1 = Block(channel)
        self.block2 = Block(96)
        self.block3 = Block(96)
        
        # Add dropout and batch normalization to GRU layers
        self.gru1 = nn.GRU(input_size=96, hidden_size=32, batch_first=True, dropout=0.2)
        self.gru2 = nn.GRU(input_size=32, hidden_size=32, batch_first=True, dropout=0.2)
        self.gru3 = nn.GRU(input_size=64, hidden_size=32, batch_first=True, dropout=0.2)
        self.gru4 = nn.GRU(input_size=96, hidden_size=32, batch_first=True, dropout=0.2)
        
        self.gru_linear = nn.Linear(225, 1)
        self.bn_linear = nn.BatchNorm1d(32)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        # Initialize the linear layers
        nn.init.xavier_normal_(self.gru_linear.weight)
        nn.init.xavier_normal_(self.fc1.weight)
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = x.permute(0, 2, 1)
        
        gru_out1, _ = self.gru1(x)
        gru_out2, _ = self.gru2(gru_out1)
        gru_out = torch.cat([gru_out1, gru_out2], dim=2)
        
        gru_out3, _ = self.gru3(gru_out)
        gru_out = torch.cat([gru_out1, gru_out2, gru_out3], dim=2)
        
        linear_out = self.dropout(self.relu(self.gru_linear(gru_out.permute(0, 2, 1))))
        gru_out4, _ = self.gru4(linear_out.permute(0, 2, 1))
        
        x = self.flatten(gru_out4)
        x = self.bn_linear(x)
        x = self.dropout(x)
        x = self.fc1(x)
        return x

class ChronoModel(LightningModule):
    def __init__(self):
        super(ChronoModel, self).__init__()
        self.model = ChronoNet(8)
        self.lr = 1e-3  # Increased learning rate
        self.bs = 32    # Reduced batch size
        self.worker = 2
        self.acc = torchmetrics.Accuracy(task="binary")
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0]))  # Balanced weight
        
        # Add early stopping metrics
        self.best_val_acc = 0.0
        self.patience_counter = 0
        
    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.1, patience=3, verbose=True
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_acc"
            }
        }

    def training_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        
        # Add gradient clipping
        torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1.0)
        
        preds = torch.sigmoid(out.flatten())
        acc = self.acc(preds, label.long().flatten())
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        
        preds = torch.sigmoid(out.flatten())
        acc = self.acc(preds, label.long().flatten())
        
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

SyntaxError: invalid syntax (818234999.py, line 12)

In [ ]:
# Initialize and train
model = ChronoModel()
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
trainer = Trainer(
    max_epochs=50,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=5),
        ModelCheckpoint(monitor='val_acc', mode='max')
    ]
)
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\zokov\.conda\envs\py3\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\zokov\.conda\envs\py3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | ChronoNet         | 131 K  | train
1 | acc       | BinaryAccuracy    | 0      | train
2 | criterion | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------
131 K     Trainable params
0         Non-trainable params
131 K     To

MisconfigurationException: `train_dataloader` must be implemented to be used with the Lightning Trainer